# HW 3

By Steven Jia

# 1. SCAD Regression

 Create your own PyTorch class that implements the method of SCAD regularization and variable selection (smoothly clipped absolute deviations) for linear models.

In [81]:
import torch # we are going to use pytorch instead of numpy because it's much faster.
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

In [2]:
device = torch.device("cpu")
dtype = torch.float64

In [318]:
class SCAD(nn.Module):
    def __init__(self, input_size, alpha=0.5, lambda_val=0.5, report_epoch = False):
        super().__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.lambda_val = lambda_val
        self.report_epoch = report_epoch

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        return self.linear(x)

    def scad_penalty(self, beta_hat, a_val, lambda_val):
        is_linear = (torch.abs(beta_hat) <= lambda_val)
        is_quadratic = torch.logical_and(lambda_val < torch.abs(beta_hat), torch.abs(beta_hat) <= a_val * lambda_val)
        is_constant = (a_val * lambda_val) < torch.abs(beta_hat)

        linear_part = lambda_val * torch.abs(beta_hat) * is_linear
        quadratic_part = (2 * a_val * lambda_val * torch.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
        constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
        return linear_part + quadratic_part + constant_part

    def loss(self, y_pred, y_true):
        mse_loss = nn.MSELoss()(y_pred, y_true)

        SCAD_loss = self.scad_penalty(self.linear.weight, self.alpha, self.lambda_val)

        loss = mse_loss + torch.sum(SCAD_loss)
        return loss

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        # print(y.shape())
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad() # reset gradient
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward() # backpropogation
            optimizer.step() # update weights

            # print progress every 10 steps
            if (epoch + 1) % 10 == 0 and self.report_epoch:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        return self.linear.weight


### Testing my SCAD implementation on a real data set

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA 441/concrete.csv')

In [8]:
data

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [96]:
MinMaxScale = MinMaxScaler()

In [310]:
x = torch.tensor(data.drop(columns=['strength']).values, device = device)
y = torch.tensor(data['strength'].values, device = device).unsqueeze(1)

#### A KFold Test

In [176]:
mse_gb = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model = SCAD(input_size=x.shape[1],alpha=0.8,lambda_val=1)

for idxtrain, idxtest in kf.split(x):
  model = SCAD(input_size=x.shape[1],alpha=0.8,lambda_val=1)
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = MinMaxScale.fit_transform(xtrain)
  xtest = MinMaxScale.transform(xtest)

  model.fit(torch.tensor(xtrain), ytrain.clone().detach().unsqueeze(1), learning_rate = 0.5)
  yhat_gb = model.predict(torch.tensor(xtest))

  mse_gb.append(nn.MSELoss()(yhat_gb, ytest.clone().detach().unsqueeze(1)))
print('The Cross-validated Mean Squared Error for Gradient Boosting is : '+str(torch.mean(torch.tensor(mse_gb))))

Epoch [10/100], Loss: 188.74084575133548
Epoch [20/100], Loss: 150.9289613303605
Epoch [30/100], Loss: 135.61881909819414
Epoch [40/100], Loss: 128.57182609444826
Epoch [50/100], Loss: 126.42173283105899
Epoch [60/100], Loss: 125.45915370140784
Epoch [70/100], Loss: 124.99603977905709
Epoch [80/100], Loss: 124.75117766678039
Epoch [90/100], Loss: 124.62661619337624
Epoch [100/100], Loss: 124.55909163422704
Epoch [10/100], Loss: 187.93611876417964
Epoch [20/100], Loss: 151.15036354973415
Epoch [30/100], Loss: 136.15851254290416
Epoch [40/100], Loss: 128.9704651344102
Epoch [50/100], Loss: 126.8626284747497
Epoch [60/100], Loss: 125.8725975918889
Epoch [70/100], Loss: 125.46229528847724
Epoch [80/100], Loss: 125.23370809968745
Epoch [90/100], Loss: 125.12664424301352
Epoch [100/100], Loss: 125.07257941424568
Epoch [10/100], Loss: 189.1659512566804
Epoch [20/100], Loss: 152.42314904211892
Epoch [30/100], Loss: 137.2319152747269
Epoch [40/100], Loss: 130.87954785199133
Epoch [50/100], Loss

#### Determining a variable selection

In [315]:
model = SCAD(input_size=x.shape[1],alpha=.235,lambda_val=1)

In [183]:
scaled_x = torch.tensor(MinMaxScale.fit_transform(x))
model.fit(scaled_x, y, learning_rate = 0.5)

Epoch [10/100], Loss: 186.0196304195578
Epoch [20/100], Loss: 148.99873979568926
Epoch [30/100], Loss: 133.7726301906199
Epoch [40/100], Loss: 126.39458231829214
Epoch [50/100], Loss: 124.99092185774028
Epoch [60/100], Loss: 122.96020927165088
Epoch [70/100], Loss: 123.08438023659951
Epoch [80/100], Loss: 122.79511927924047
Epoch [90/100], Loss: 122.04032590027713
Epoch [100/100], Loss: 121.96995711983931


In [184]:
model.get_coefficients()

Parameter containing:
tensor([[41.5320, 25.2498,  9.3019, -0.1909, 25.5461,  3.5289,  1.4021, 34.0545]],
       dtype=torch.float64, requires_grad=True)

Based on my SCAD implementation, the most important features for predicting concrete strength are the cement, age, slag, and superplastic values.

# 2. Simulation Design

 Generate 500 data sets where the input features have a strong correlation structure (you may consider a 0.9) and apply ElasticNet, SqrtLasso and SCAD to check which method produces the best approximation of an ideal solution, such as a "betastar" you design with a sparsity pattern.

### ElasticNet and SQRTLasso

In [308]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5, report_epoch = False):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(ElasticNet, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.report_epoch = report_epoch

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        # norm be thought of as a distance function but generalized to any number of features
        l1_reg = torch.norm(self.linear.weight, p=1) # p specifies the exponent in the norm: sum(abs(x)**p)**(1./p)
        l2_reg = torch.norm(self.linear.weight, p=2)

        loss = mse_loss + self.alpha * (
            self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1/2) * l2_reg**2
        )
        return loss

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad() # reset gradient
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward() # backpropogation
            optimizer.step() # update weights

            # print progress every 10 steps
            if (epoch + 1) % 10 == 0 and self.report_epoch:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight


In [276]:
class SqrtLasso(nn.Module):
    def __init__(self, input_size, alpha=0.1, report_epoch = False):
        """
        Initialize the  regression model.


        """
        super(SqrtLasso, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.report_epoch = report_epoch

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1,dtype=torch.float64)
        # l2_reg = torch.norm(self.linear.weight, p=2,dtype=torch.float64)

        loss = torch.sqrt(mse_loss) + self.alpha * (l1_reg)

        return loss

    def fit(self, X, y, num_epochs=200, learning_rate=0.01):
        """
        Fit the model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0 and self.report_epoch:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

### Generating Data Sets

In [277]:
from scipy.linalg import toeplitz
import numpy as np

In [278]:
# we want to define a function for generating x with a prescribed number of observations, features and Toeplitz correlation structure.
def make_correlated_features(num_samples,p,rho):
  vcor = []
  for i in range(p):
    vcor.append(rho**i)
  r = toeplitz(vcor)
  mu = np.repeat(0,p)
  x = np.random.multivariate_normal(mu, r, size=num_samples)
  return x

In [325]:
rho = 0.9
p = 20
n = 150

In [280]:
datasets = []
for x in range(0, 500):
  datasets.append(torch.tensor(make_correlated_features(n,p,rho)))

In [326]:
beta = np.array([5,0,8,0,-3,0,1,0,7,0]) # self-defined sparsity pattern
beta = beta.reshape(-1,1)
betastar = np.concatenate([beta,np.repeat(0,p-len(beta)).reshape(-1,1)],axis=0) # any remaining betas are 0

In [282]:
# generate response with noise
reponses = []
for x in datasets:
  reponses.append((x@betastar + 1.5*np.random.normal(size=(n,1))).clone().detach())

In [320]:
SCAD_m = SCAD(input_size=p, alpha=0.5, lambda_val=0.5)
EN = ElasticNet(input_size=p, alpha=1.0, l1_ratio=0.5)
SQRT = SqrtLasso(input_size=p, alpha=0.1)

In [322]:
SCAD_pred_error = np.repeat(0,p)
EN_pred_error = np.repeat(0,p)
SQRT_pred_error = np.repeat(0,p)

for x in range(0, 500):
  SCAD_m.fit(datasets[x], reponses[x])
  SCAD_pred_error = np.add(SCAD_pred_error, np.subtract(betastar.flatten(), SCAD_m.get_coefficients().detach().numpy()))
  EN.fit(datasets[x], reponses[x])
  EN_pred_error = np.add(EN_pred_error, np.subtract(betastar.flatten(), EN.get_coefficients().detach().numpy()))
  SQRT.fit(datasets[x], reponses[x])
  SQRT_pred_error = np.add(SQRT_pred_error, np.subtract(betastar.flatten(), SQRT.get_coefficients().detach().numpy()))

In [323]:
print("Average difference between predicted beta and actual beta for SCAD: " + str(np.sum(np.abs(SCAD_pred_error)) / 500))
print("Average difference between predicted beta and actual beta for ElasticNet: " + str(np.sum(np.abs(EN_pred_error)) / 500))
print("Average difference between predicted beta and actual beta for SQRT Lasso: " + str(np.sum(np.abs(SQRT_pred_error)) / 500))

Average difference between predicted beta and actual beta for SCAD: 2.0317050538402137
Average difference between predicted beta and actual beta for ElasticNet: 20.20477584925671
Average difference between predicted beta and actual beta for SQRT Lasso: 4.325177263745284


Based on my test with 500 datasets, SCAD does the best at approximating the ideal solution. I used default hyperparameter values and a sparsity pattern of 5 significant features out of 20 total features.